# Percussion Detection

Trying to classify percussion samples...

In [128]:
import sys
sys.path.append('..')

In [129]:
import warnings
warnings.simplefilter('ignore', RuntimeWarning)
warnings.simplefilter('ignore', UserWarning)

In [130]:
import IPython.display as display
import librosa
import matplotlib.pyplot as plot
import numpy as np

from philharmonia import dataframe, datafile

In [131]:
import pyloudnorm

meter = None

def loudness(sample):

    x, sr = librosa.load(datafile(sample.file), sr=None, mono=True)

    global meter
    meter = meter or pyloudnorm.Meter(sr, block_size=100e-3)

    db = meter.integrated_loudness(x)

    return int(db)

In [132]:
def flatness(sample):

    x, sr = librosa.load(datafile(sample.file), sr=None, mono=True)

    magn = np.abs(np.fft.rfft(x * np.hanning(x.size))) ** 2

    prod = np.exp(np.mean(np.log(magn)))
    mean = np.mean(magn)

    flat = 20 * np.log10(prod / mean)

    return round(flat, 2)

In [133]:
def spread(sample):

    x, sr = librosa.load(datafile(sample.file), sr=None, mono=True)

    magn = np.abs(np.fft.rfft(x * np.hanning(x.size))) ** 2
    freq = np.fft.rfftfreq(x.size) * sr
    cent = np.sum(freq * magn) / np.sum(magn)

    freq = (freq - cent) ** 2
    sprd = np.sqrt(np.sum(freq * magn) / np.sum(magn))

    return round(sprd, 2)

In [134]:
def centroid(sample, relative=False):

    x, sr = librosa.load(datafile(sample.file), sr=None, mono=True)

    magn = np.abs(np.fft.rfft(x * np.hanning(x.size))) ** 2
    freq = np.fft.rfftfreq(x.size) * sr
    cent = np.sum(freq * magn) / np.sum(magn)

    if relative:

        freq = (freq - cent) ** 2
        cent = np.sqrt(np.sum(freq * magn) / np.sum(magn))

    return round(cent, 2)

In [135]:
query = [
    "(percussion == False & style == 'normal' & length == '1')",
    "(percussion == True)",
]

filter = ['file', 'instrument', 'percussion']
sort = ['percussion', 'instrument', 'loudness', 'spread']

data = dataframe().query(' | '.join(query)).filter(filter)

data['loudness'] = data.apply(lambda sample: loudness(sample), axis=1)
data['flatness'] = data.apply(lambda sample: flatness(sample), axis=1)
data['spread'] = data.apply(lambda sample: spread(sample), axis=1)
data['centroid'] = data.apply(lambda sample: centroid(sample), axis=1)

data = data.sort_values(sort)

In [150]:
from sklearn.svm import LinearSVC as SVM

X = data[['flatness', 'spread', 'centroid']].to_numpy()
y = data['percussion'].to_numpy().astype(float)

svm = SVM(class_weight='balanced', dual=(X.shape[0] <= X.shape[1]), random_state=0).fit(X, y)

p = svm.predict(X)
e = (p != y).astype(int)

results = data \
    .assign(error=e) \
    .groupby(['percussion', 'instrument']) \
    .agg(total=('instrument', len), errors=('error', sum)) \
    .reset_index() \
    .filter(['percussion', 'instrument', 'total', 'errors'])

display.HTML(results.to_html())

,percussion,instrument,total,errors
0,False,bass-clarinet,208,2
1,False,bassoon,169,2
2,False,clarinet,187,1
3,False,contrabassoon,158,15
4,False,english-horn,162,2
5,False,flute,186,101
6,False,french-horn,119,0
7,False,oboe,134,0
8,False,saxophone,165,6
9,False,trombone,175,35
